###  Imports

In [1]:
import torch
from model import create_model
from utils import FeatureExtractor

import config

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

from dataset import FiftyOneTorchDataset, get_transforms
from model import create_model
from utils import add_detections

from engine import train_model
import config

torch.manual_seed(1)

In [2]:
print(fo.list_datasets())

['coco-2017-validation']


In [3]:
dataset = fo.load_dataset("coco-2017-validation")
dataset.delete()

In [4]:
#Load in the dataset from the FiftyOne model Zoo
fo_dataset = foz.load_zoo_dataset("coco-2017", "validation")

#needed to calculate image height and width
fo_dataset.compute_metadata()

session = fo.launch_app(fo_dataset)


# to filter certain items from the dataset we can
item_list = ["car", "dog", "bus", 'fork', 'tie', 'person']
item_list = ['bus', 'dog']
item_view = fo_dataset.filter_labels("ground_truth",
        F("label").is_in(item_list))

session.view = item_view

# split the dataset in train and test set
train_view = item_view.take((len(item_view) * config.TRAIN_TEST_SPLIT), seed=51)
test_view = item_view.exclude([s.id for s in train_view])

print(f'Traning on {len(train_view)} samples')
print(f'Testing on {len(test_view)} samples')


train_transforms, test_transforms = get_transforms()

# use our dataset and defined transformations
train_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=item_list)
evaluation_dataset = FiftyOneTorchDataset(test_view, test_transforms,
        classes=item_list)

Found annotations at 'C:\Users\blain\fiftyone\coco-2017\raw\instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [21.2s elapsed, 0s remaining, 249.5 samples/s]      
Dataset 'coco-2017-validation' created
Connected to FiftyOne on port 5151 at 127.0.0.1.
If you are not connecting to a remote session, you may need to start a new session and specify a port


Traning on 292 samples
Testing on 73 samples


# Load the base FRCNN 

In [7]:
MODEL_TYPE = 'CLIP-Backbone-FRCNN'
CHECKPOINT_NAME = f'{MODEL_TYPE}_epoch_28.pth'

if item_list[0] != 'background':
     item_list.insert(0,'background')

frcnn_model = create_model(MODEL_TYPE, classes=item_list)
checkpoint = torch.load(CHECKPOINT_NAME)
frcnn_model = create_model(MODEL_TYPE, classes=item_list)

frcnn_model.load_state_dict(checkpoint)
frcnn_model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): FeatureExtractor(
    (model): ModifiedResNet(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (relu): ReLU(inplace=True)
      (layer1): Sequential(
        (0): Bottleneck(
          (con

# Load the CLIP FRCNN

In [6]:
# test out the trained CLIP-FRCNN
MODEL_TYPE = 'CLIP-FRCNN'
CHECKPOINT_NAME = f'{MODEL_TYPE}_epoch_68.pth'


# tokenize item list for CLIP
import clip
_, preprocess = clip.load("RN50", device=config.DEVICE)

if item_list[0] != 'background':
     item_list.insert(0,'background')

text_tokens = clip.tokenize(["This is " + desc for desc in item_list]).cuda()


clip_frcnn_model = create_model(MODEL_TYPE, classes=text_tokens)
checkpoint = torch.load(CHECKPOINT_NAME)
clip_frcnn_model = create_model(MODEL_TYPE, classes=text_tokens)

clip_frcnn_model.load_state_dict(checkpoint)
clip_frcnn_model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): FeatureExtractor(
    (model): ModifiedResNet(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (relu): ReLU(inplace=True)
      (layer1): Sequential(
        (0): Bottleneck(
          (con

# Check Faster RCNN performance

In [8]:
add_detections(frcnn_model, evaluation_dataset, fo_dataset, field_name="frcnn_predictions")

results = fo.evaluate_detections(
    test_view,
    "frcnn_predictions",
    classes=item_list,
    eval_key="eval",
    compute_mAP=True
)
session.view = item_view
print(f'mAP: {results.mAP()}')
results.print_report()

Using device cuda
   0% ||------------------|  0/73 [37.1ms elapsed, ? remaining, ? samples/s] 

C:\Users\blain\anaconda3\envs\torch-frcnn\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 100% |███████████████████| 73/73 [7.1s elapsed, 0s remaining, 12.6 samples/s]      
Evaluating detections...
 100% |███████████████████| 73/73 [637.0ms elapsed, 0s remaining, 114.6 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 73/73 [726.1ms elapsed, 0s remaining, 100.5 samples/s]     


# Check CLIP FRCNN performance

In [7]:
add_detections(clip_frcnn_model, evaluation_dataset, fo_dataset, field_name="clip_frcnn_predictions")

results = fo.evaluate_detections(
    test_view,
    "clip_frcnn_predictions",
    classes=item_list,
    eval_key="clip_eval",
    compute_mAP=True
)
session.view = item_view
print(f'mAP: {results.mAP()}')
results.print_report()

session.view = item_view
print(f'mAP: {results.mAP()}')
results.print_report()

Using device cuda


C:\Users\blain\anaconda3\envs\torch-frcnn\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 100% |███████████████████| 73/73 [15.6s elapsed, 0s remaining, 5.2 samples/s]      
Evaluating detections...
 100% |███████████████████| 73/73 [2.7s elapsed, 0s remaining, 29.3 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 73/73 [4.4s elapsed, 0s remaining, 18.1 samples/s]      


mAP: 0.003243333724199572
              precision    recall  f1-score   support

  background       0.00      0.00      0.00         0
         bus       0.01      0.67      0.03        83
         dog       0.01      0.87      0.02        39

   micro avg       0.01      0.74      0.03       122
   macro avg       0.01      0.52      0.02       122
weighted avg       0.01      0.74      0.03       122



mAP: 0.003243333724199572
              precision    recall  f1-score   support

  background       0.00      0.00      0.00         0
         bus       0.01      0.67      0.03        83
         dog       0.01      0.87      0.02        39

   micro avg       0.01      0.74      0.03       122
   macro avg       0.01      0.52      0.02       122
weighted avg       0.01      0.74      0.03       122



# Create a CLIP enabled model



In [13]:
frcnn_model.roi_heads.score_thresh

0.05

In [9]:
frcnn_model.roi_heads.detections_per_img

100

In [ ]:
# test out the trained CLIP-FRCNN
MODEL_TYPE = 'CLIP-FRCNN'
CHECKPOINT_NAME = f'{MODEL_TYPE}_epoch_68.pth'

if item_list[0] != 'background':
     item_list.insert(0,'background')

clip_frcnn_model = create_model(MODEL_TYPE, classes=item_list)
checkpoint = torch.load(CHECKPOINT_NAME)
clip_frcnn_model = create_model(MODEL_TYPE, classes=item_list)

clip_frcnn_model.load_state_dict(checkpoint)
clip_frcnn_model.eval()

In [23]:
#TEST out the trained Faster RCNN replacing the box head and box predictors

from torch.cuda.amp import autocast
from model import CLIPHead, CLIPRCNNPredictor

frcnn_model.roi_heads.box_head = CLIPHead()
frcnn_model.roi_heads.box_predictor = CLIPRCNNPredictor(1024, text_tokens)  #CLIP embeds into 1024 dimensions for the RN50 implementation

frcnn_model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): FeatureExtractor(
    (model): ModifiedResNet(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (relu): ReLU(inplace=True)
      (layer1): Sequential(
        (0): Bottleneck(
          (con

In [26]:
# test requiring the rpn being more confident
frcnn_model.roi_heads.score_thresh = .99

In [27]:
with autocast():
    add_detections(frcnn_model, evaluation_dataset, fo_dataset, field_name="clip_predictions")

results = fo.evaluate_detections(
    test_view,
    "clip_predictions",
    classes=item_list,
    eval_key="clip_eval",
    compute_mAP=True
)

Using device cuda
   1% |-------------------|  1/73 [330.7ms elapsed, 23.7s remaining, 4.8 samples/s] 


IndexError: list index out of range

In [ ]:
session.view = item_view
print(f'mAP: {results.mAP()}')
results.print_report()

In [22]:
for i in range(100):
    test_image = [torch.rand(3, 224, 224).cuda()]
    with autocast():
        out = frcnn_model(test_image)
    print(out[0]['labels'].shape)

torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Size([0])
torch.Si